In [2]:
import ipywidgets as widgets  # Widgets for creating interactive UI elements.
from IPython.display import display, clear_output  # Display and clear output in Jupyter notebooks.
import sqlite3  # For interacting with SQLite databases.
import pandas as pd  # Data manipulation and analysis.
import matplotlib.pyplot as plt  # For creating visualizations.
from tabulate import tabulate  # For displaying data in a formatted table.

# Function 1: Analyze artist popularity
def analyze_artist_popularity(db_file):
    artist_name = input_widget.value.strip()  # Get the artist's name from the input widget.
    output.clear_output()  # Clear any existing output.
    with output:
        conn = sqlite3.connect(db_file)  # Connect to the database.
        query = """
            SELECT g.Genre AS Genre, 
                   AVG(CASE WHEN s.Artist_ID = a.ID THEN s.Popularity END) AS Artist_Popularity,
                   AVG(s.Popularity) AS Overall_Popularity
            FROM Song s
            JOIN Genre g ON s.Genre_ID = g.ID
            JOIN Artist a ON s.Artist_ID = a.ID
            WHERE a.Artist_Name = ?
            GROUP BY g.Genre;
        """  # SQL query to fetch artist and genre popularity data.
        df = pd.read_sql_query(query, conn, params=(artist_name,))  # Execute query and load into a DataFrame.
        conn.close()  # Close the database connection.

        if df.empty:  # Check if the query returned any data.
            print(f"No data found for artist: {artist_name}")  # Display a message if no data is found.
            return

        print(tabulate(df, headers='keys', tablefmt='psql', showindex=False))  # Display data as a formatted table.

        # Plot chart
        plt.figure(figsize=(10, 6))  # Set figure size.
        plt.bar(df['Genre'], df['Artist_Popularity'], label=f"{artist_name} Popularity", color='blue', alpha=0.7)  # Artist popularity bars.
        plt.bar(df['Genre'], df['Overall_Popularity'], label='Overall Popularity', color='orange', alpha=0.5)  # Overall popularity bars.
        plt.xticks(rotation=45)  # Rotate x-axis labels for readability.
        plt.title(f"Popularity of {artist_name} Across Genres")  # Chart title.
        plt.xlabel("Genre")  # Label for x-axis.
        plt.ylabel("Popularity")  # Label for y-axis.
        plt.legend()  # Add a legend.
        plt.tight_layout()  # Adjust layout to fit everything.
        plt.show()  # Display the chart.

# Function 2: Top 5 artists
def calculate_top5_artists(db_file):
    try:
        start_year = int(start_year_widget.value)  # Get start year from widget input.
        end_year = int(end_year_widget.value)  # Get end year from widget input.
    except ValueError:  # Handle invalid numeric input.
        with output:
            print("Invalid year range. Please enter numeric values.")  # Display error message.
        return

    if not (1998 <= start_year <= 2020 and 1998 <= end_year <= 2020 and start_year <= end_year):  # Validate year range.
        with output:
            print("Invalid year range. Please ensure 1998 <= start year <= end year <= 2020.")  # Display validation error.
        return

    output.clear_output()  # Clear any existing output.
    with output:
        conn = sqlite3.connect(db_file)  # Connect to the database.
        query = '''
            SELECT Artist.Artist_Name AS Artist,
                   COUNT(Song.ID) AS Total_Songs,
                   AVG(Song.Popularity) AS Avg_Popularity,
                   (COUNT(Song.ID) * 0.6 + AVG(Song.Popularity) * 0.4) AS Rank_Value
            FROM Song
            JOIN Artist ON Song.Artist_ID = Artist.ID
            WHERE Song.Year BETWEEN ? AND ?
            GROUP BY Artist.Artist_Name
            ORDER BY Rank_Value DESC
            LIMIT 5;
        '''  # SQL query to fetch top 5 artists by rank value.
        df = pd.read_sql_query(query, conn, params=(start_year, end_year))  # Execute query and load into a DataFrame.
        conn.close()  # Close the database connection.

        if df.empty:  # Check if the query returned any data.
            print(f"No data found for the years {start_year} to {end_year}.")  # Display a message if no data is found.
            return

        print(tabulate(df, headers='keys', tablefmt='psql', showindex=False))  # Display data as a formatted table.

        # Plot chart
        plt.figure(figsize=(10, 6))  # Set figure size.
        plt.bar(df['Artist'], df['Rank_Value'], color='skyblue', alpha=0.7)  # Plot rank values as bars.
        plt.title(f"Top 5 Artists by Rank Value ({start_year}-{end_year})")  # Chart title.
        plt.xlabel("Artists")  # Label for x-axis.
        plt.ylabel("Rank Value")  # Label for y-axis.
        plt.xticks(rotation=45)  # Rotate x-axis labels for readability.
        plt.tight_layout()  # Adjust layout to fit everything.
        plt.show()  # Display the chart.

# Widgets
input_widget = widgets.Text(  # Text input widget for artist name.
    description="Artist:",
    placeholder="Enter artist's name",
    style={'description_width': 'initial'}
)

start_year_widget = widgets.Text(  # Text input widget for start year.
    description="Start Year:",
    placeholder="1998",
    style={'description_width': 'initial'}
)

end_year_widget = widgets.Text(  # Text input widget for end year.
    description="End Year:",
    placeholder="2020",
    style={'description_width': 'initial'}
)

output = widgets.Output()  # Output widget to display results and messages.

analyze_button = widgets.Button(description="Analyze Artist Popularity")  # Button for analyzing artist popularity.
top5_button = widgets.Button(description="Top 5 Artists")  # Button for fetching top 5 artists.

# Event handlers
analyze_button.on_click(lambda b: analyze_artist_popularity("CWDatabase.db"))  # Attach event handler to analyze button.
top5_button.on_click(lambda b: calculate_top5_artists("CWDatabase.db"))  # Attach event handler to top 5 button.

# Layout
menu = widgets.VBox([  # Arrange widgets vertically in a box.
    widgets.HTML("<h2>Music Data Analysis Menu</h2>"),  # Menu header.
    widgets.HTML("<h3>Analyze Artist Popularity</h3>"),  # Section header for artist popularity.
    input_widget,  # Artist input widget.
    analyze_button,  # Analyze button.
    widgets.HTML("<h3>Top 5 Artists</h3>"),  # Section header for top 5 artists.
    start_year_widget,  # Start year input widget.
    end_year_widget,  # End year input widget.
    top5_button,  # Top 5 button.
    output  # Output display area.
])

display(menu)  # Display the entire menu.

